Escolher Variaveis que Impactam o tempo de preparo para fazer um modelo, que preve o tempo de preparo!

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Carregar a base
df = pd.read_excel('../Data/dados_amr.xlsx', sheet_name='Base_Limpa')

# Variáveis categóricas para analisar
categoricas = ['modo_envio', 'segmento', 'região', 'categoria', 'subcategoria', 'estado', 'dia_semana_pedido']

# Rodar ANOVA para cada uma
for var in categoricas:
    print(f'\nAnalisando {var}...')
    model = smf.ols(f'tempo_preparacao ~ C({var})', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)




Analisando modo_envio...
                     sum_sq      df            F  PR(>F)
C(modo_envio)  20734.388219     3.0  7056.889125     0.0
Residual        9784.128891  9990.0          NaN     NaN

Analisando segmento...
                   sum_sq      df         F    PR(>F)
C(segmento)     11.098147     2.0  1.817289  0.162519
Residual     30507.418964  9991.0       NaN       NaN

Analisando região...
                 sum_sq      df         F    PR(>F)
C(região)     29.567010     4.0  2.421737  0.046116
Residual   30488.950101  9989.0       NaN       NaN

Analisando categoria...
                    sum_sq      df         F    PR(>F)
C(categoria)     11.048285     2.0  1.809121  0.163852
Residual      30507.468825  9991.0       NaN       NaN

Analisando subcategoria...
                       sum_sq      df         F    PR(>F)
C(subcategoria)     58.508472    16.0  1.197757  0.260496
Residual         30460.008638  9977.0       NaN       NaN

Analisando estado...
                sum_sq   

 Variáveis Significativas (PR(>F) < 0.05):
 - região
 - estado
 - dia_semana_pedido
 - modo_envio

In [28]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_excel('../Data/dados_amr.xlsx')

# Definir target e features
y = df['tempo_preparacao']
X = df[['modo_envio', 'região', 'dia_semana_pedido']]
X = df.drop(columns=["tempo_preparacao"])

# Codificar variáveis categóricas
X_encoded = pd.get_dummies(X, drop_first=True)

# Dividir em treino, validação e teste
X_temp, X_test, y_temp, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Treinar o modelo
model = HistGradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

# Avaliação na validação
y_val_pred = model.predict(X_val)
print("Validação")
print("MSE:", mean_squared_error(y_val, y_val_pred))
print("R²:", r2_score(y_val, y_val_pred))

# Avaliação no teste final
y_test_pred = model.predict(X_test)
print("\nTeste Final")
print("MSE:", mean_squared_error(y_test, y_test_pred))
print("R²:", r2_score(y_test, y_test_pred))

Validação
MSE: 0.12884367815050538
R²: 0.957850662634745

Teste Final
MSE: 0.10904849162435548
R²: 0.962476374831337


In [3]:
import joblib
joblib.dump(model, 'modelo_tempo.pkl')
modelo_carregado = joblib.load('modelo_tempo.pkl')

In [4]:
correlation = df['tempo_preparacao'].corr(df['lucro'])
print("Correlação de Pearson:", correlation)

Correlação de Pearson: -0.004649353144150638
